In [1]:
import pandas as pd
import geopandas as gpd
import shapely
from scipy.stats import pearsonr
import numpy as np
from libpysal import weights
import time

from sklearn.metrics import mean_squared_error as mse
from sklearn.ensemble import RandomForestRegressor
from sklearn.datasets import make_classification

import os
from itertools import zip_longest



In [2]:
def grouper(iterable_obj, count, fillvalue=None):
    args = [iter(iterable_obj)] * count
    return zip_longest(*args, fillvalue=fillvalue)

In [3]:
nadf = pd.DataFrame(columns = ["data2015", "residential2015","data2016", "residential2016","data2017", 
                     "residential2017","data2018", "residential2018","data2019", "residential2019"])

In [4]:
import warnings
warnings.filterwarnings('ignore')

In [5]:
def clean_it(PATH):
    for filename in grouper(os.listdir(PATH), 5, np.NaN):
        print(filename)
        
        data2015 = pd.read_csv(PATH + filename[0])
        data2016 = pd.read_csv(PATH + filename[1])
        data2017 = pd.read_csv(PATH + filename[2])
        data2018 = pd.read_csv(PATH + filename[3])
        data2019 = pd.read_csv(PATH + filename[4])
        
        data2015.rename(columns = {"p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
        data2016.rename(columns = {"p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
        data2017.rename(columns = {"p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
        data2018.rename(columns = {"p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
        data2019.rename(columns = {"p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
        
        data2015 = data2015.replace({-99997 : np.NaN, np.inf : np.NaN})
        data2016 = data2016.replace({-99997 : np.NaN, np.inf : np.NaN})
        data2017 = data2017.replace({-99997 : np.NaN, np.inf : np.NaN})
        data2018 = data2018.replace({-99997 : np.NaN, np.inf : np.NaN})
        data2019 = data2019.replace({-99997 : np.NaN, np.inf : np.NaN})

        b15 = data2015[data2015.woonfunctiesum>0]
        b16 = data2016[data2016.woonfunctiesum>0]
        b17 = data2017[data2017.woonfunctiesum>0]
        b18 = data2018[data2018.woonfunctiesum>0]
        b19 = data2019[data2019.woonfunctiesum>0]

        # set percentage lower than 0 to na
        b15[b15["p_benefits_g"] < 0] = np.NaN 
        b16[b16["p_benefits_g"] < 0] = np.NaN 
        b17[b17["p_benefits_g"] < 0] = np.NaN 
        b18[b18["p_benefits_g"] < 0] = np.NaN 
        b19[b19["p_benefits_g"] < 0] = np.NaN 

        def grid_filler(data, grid_column, buurt_column):
            print(grid_column + " missing before: {}".format(data[grid_column].isna().sum()))
            missing_grid_data = data[grid_column].isna().index.values
            data.loc[missing_grid_data, grid_column] = data.loc[missing_grid_data, buurt_column]
            print(grid_column + " missing after: {}".format(data[grid_column].isna().sum()))
            return data[grid_column]

        grid_variables = []
        neighborhood_variables = []
        for var in data2015.columns:
            if var.endswith("_g"):
                grid_variables.append(var)
            elif var.endswith("_b"):
                neighborhood_variables.append(var)

        grid_variables.sort(key=str.lower)
        neighborhood_variables.sort(key=str.lower)

        ls = list()
        for i, j in zip(grid_variables, neighborhood_variables):
            ls.append((i,j))
        print(ls)

        for i in ls:
            data2015[i[0]] = grid_filler(data2015, i[0], i[1])
            data2016[i[0]] = grid_filler(data2016, i[0], i[1])
            data2017[i[0]] = grid_filler(data2017, i[0], i[1])
            data2018[i[0]] = grid_filler(data2018, i[0], i[1])
            data2019[i[0]] = grid_filler(data2019, i[0], i[1])

        var = ['geometry', 'bijeenkomstfunctiesum',
    #        'celfunctie', 
               'gezondheidszorgfunctiesum', 'industriefunctiesum',
           'kantoorfunctiesum', 'logiesfunctiesum', 'onderwijsfunctiesum', 'sportfunctiesum',
           'winkelfunctiesum', 'woonfunctiesum', 'oppervlakteVerblijfsobjectmean', 'BU_CODE',
           'y', 'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
           'inw_4564_g', 'inw_65_g', 'p_NL', 'p_western_g', 'p_nonWestern_g',
           'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
           'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
           'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
           'afs_transit_g', 'mean_WOZ', 
            'PLaagste40Inkomen', 'PHoogste20Inkomen','bouwjaarmean', 'bouwjaarmin', 'bouwjaarmax', 'y-1']
        t15 = data2015[var]
        t16 = data2016[var]
        t17 = data2017[var]
        t18 = data2018[var]
        t19 = data2019[var]


        t15["n_missing"] = t15.isna().sum(axis=1)
        t16["n_missing"] = t16.isna().sum(axis=1)
        t17["n_missing"] = t17.isna().sum(axis=1)
        t18["n_missing"] = t18.isna().sum(axis=1)
        t19["n_missing"] = t19.isna().sum(axis=1)

        t15b = t15[(t15.woonfunctiesum > 0) & (t15.n_missing < 15)]
        t16b = t16[(t16.woonfunctiesum > 0) & (t16.n_missing < 15)]
        t17b = t17[(t17.woonfunctiesum > 0) & (t17.n_missing < 15)]
        t18b = t18[(t18.woonfunctiesum > 0) & (t18.n_missing < 15)]
        t19b = t19[(t19.woonfunctiesum > 0) & (t19.n_missing < 15)]

        t15b = gpd.GeoDataFrame(t15b)
        t15b.geometry = t15b.geometry.apply(shapely.wkt.loads)

        t16b = gpd.GeoDataFrame(t16b)
        t16b.geometry = t16b.geometry.apply(shapely.wkt.loads)

        t17b = gpd.GeoDataFrame(t17b)
        t17b.geometry = t17b.geometry.apply(shapely.wkt.loads)

        t18b = gpd.GeoDataFrame(t18b)
        t18b.geometry = t18b.geometry.apply(shapely.wkt.loads)

        t19b = gpd.GeoDataFrame(t19b)
        t19b.geometry = t19b.geometry.apply(shapely.wkt.loads)

        var2 = ['bijeenkomstfunctiesum', 'geometry',
    #        'celfunctie', 
                'gezondheidszorgfunctiesum', 'industriefunctiesum',
           'kantoorfunctiesum', 'logiesfunctiesum', 'onderwijsfunctiesum', 'sportfunctiesum',
           'winkelfunctiesum', 'woonfunctiesum', 'oppervlakteVerblijfsobjectmean', 
            'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
           'inw_4564_g', 'inw_65_g', 'p_western_g', 'p_nonWestern_g',
           'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
           'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
           'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
           'afs_transit_g', 'mean_WOZ', 
            'PLaagste40Inkomen', 'PHoogste20Inkomen', 'bouwjaarmean', 'bouwjaarmin', 'bouwjaarmax', 'y-1']

        t15b = t15b[var2]
        t16b = t16b[var2]
        t17b = t17b[var2]
        t18b = t18b[var2]
        t19b = t19b[var2]

        def fill_final_na(data):
            t0 = time.time()
            total_na = data.isna().sum().sum()
            all_geom = data[["geometry", "sportfunctiesum"]]

            data = data.drop(["geometry"], axis = 1)
            incomplete = data[data.isnull().any(axis=1)]
            complete = gpd.GeoDataFrame(data[~data.isnull().any(axis=1)]) # store the rows that have no na values



            # create list of combinatoins of missing columns
            inc_list = list()
            for _, i in incomplete.iterrows():
                if not tuple(i.index[i.isna()]) in inc_list:
                    inc_list.append(tuple(i.index[i.isna()]))

        #   Create weightsmatrix and transform to row_standardized            
        #     w_all = weights.distance.KNN.from_dataframe(all_geom, k = 4, silence_warnings = True)
        #     w_all.transform = 'r'

            
            columns = data.columns

            # loop over the missing column combinations
            def train_creater(inp, w, y, test = False):
                """
                returns: new dataframe including the neighbors data
                """
                # if test: create weight matrix from each to be predicted item to known y's
                if test:
                    neighbors = []
                    for i in inp.index:
                        # w that contains the test to nearest ful y's
                        w =  weights.distance.KNN.from_dataframe(all_geom.loc[np.append(y.index.values,(i))], k = 4, silence_warnings = True)
                        neighbors.append(w.neighbors[i])
                else:
#                 Get neigbors per index
                    neighbors = [ w.neighbors[k] for k in inp.index.values.tolist()]
               
                if test:

                    rows = [ np.concatenate(y.loc[j].values) for j in neighbors] 
                    distances = [list(all_geom.loc[neigh].distance(all_geom.loc[inp.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]
                    
                else:
                    # get rows of the neigbors, tranform to np and concatenate them
                    rows = [ np.concatenate(y.loc[j].values) for j in neighbors] 
                    distances = [list(all_geom.loc[neigh].distance(all_geom.loc[inp.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]


                return pd.concat([pd.DataFrame(rows).set_index(inp.index), inp, pd.DataFrame(distances).set_index(inp.index) ], axis=1)

            for i, cols in enumerate(inc_list):
                cols = list(cols)
                x_train = complete.drop(cols, axis = 1) # only keep the cols to trained upon
                y_train = complete[cols] # y has the na values to be predicted

                w_train  = weights.distance.KNN.from_dataframe(all_geom.loc[x_train.index], k = 4, silence_warnings = True)
                w_train.transform = 'r'

                X_train = train_creater(x_train, w_train, y_train)
                # Voor x predict only keep the rows where there are only NA's in cols
                pred = incomplete[(incomplete[cols].isna().all(axis = 1) & (incomplete[columns.drop(cols)].notna().all(axis = 1)))]
                X_pred = pred.drop(cols, axis = 1)
    
                # create prediction weights of missing rows as well as known rows
                w_pred =  weights.distance.KNN.from_dataframe(all_geom.loc[X_train.index.append(X_pred.index)], k = 4, silence_warnings = True)
                w_pred.transform = 'r'
                X_pred = train_creater(X_pred, w_pred, y_train, True)
                # loop over each missing variable in cols and create model per 
                for col in cols:

                    clf = RandomForestRegressor(max_depth=6, random_state=0)
                    clf.fit(X_train, y_train[col].values.ravel())    
                    y_pred = clf.predict(X_pred)
                    incomplete.loc[X_pred.index, col] = y_pred

                print("{} na's left of original {}, {:.2f}% of cols done, time spend: {:.0f}".format(incomplete.isna().sum().sum(),  total_na, i / len(inc_list) * 100, time.time() - t0))
            print("total time: {:.0f}".format(time.time() - t0))
            return pd.concat([incomplete, complete])

        t15b = fill_final_na(t15b)
        t16b = fill_final_na(t16b)
        t17b = fill_final_na(t17b)
        t18b = fill_final_na(t18b)
        t19b = fill_final_na(t19b)

        t15b = t15b.join(data2015[["C28992R100", "geometry", "y", "BU_CODE"]])
        t16b = t16b.join(data2016[["C28992R100", "geometry", "y", "BU_CODE"]])
        t17b = t17b.join(data2017[["C28992R100", "geometry", "y", "BU_CODE"]])
        t18b = t18b.join(data2018[["C28992R100", "geometry", "y", "BU_CODE"]])
        t19b = t19b.join(data2019[["C28992R100", "geometry", "y", "BU_CODE"]])

        t15b.to_csv("../Data/filled/2015/" + filename[0] + "_filled_.csv")
        t16b.to_csv("../Data/filled/2016/" + filename[1] + "_filled_.csv")
        t17b.to_csv("../Data/filled/2017/" + filename[2] + "_filled_.csv")
        t18b.to_csv("../Data/filled/2018/" + filename[3] + "_filled_.csv")
        t19b.to_csv("../Data/filled/2019/" + filename[4] + "_filled_.csv")


In [8]:
a = clean_it("../Data/snapshot/")

('Leeuwardenvbo.csv15.csv', 'Leeuwardenvbo.csv16.csv', 'Leeuwardenvbo.csv17.csv', 'Leeuwardenvbo.csv18.csv', 'Leeuwardenvbo.csv19.csv')
[('afs_haprak_g', 'afs_huisarts_b'), ('afs_oprit_g', 'afs_oprit_b'), ('afs_superm_g', 'afs_supermarkt_b'), ('afs_train_g', 'afs_train_b'), ('afs_transit_g', 'afs_transit_b'), ('afs_vo_g', 'afs_vo_b'), ('afs_ziek_g', 'afs_ziekenhuis_b'), ('avg_electricity_g', 'avg_electicity_b'), ('avg_gas_g', 'avg_gas_b'), ('geboorte_g', 'birth_b'), ('inw_014_g', 'inw14_b'), ('inw_1524_g', 'inw24_b'), ('inw_2544_g', 'inw44_b'), ('inw_4564_g', 'inw64_b'), ('inw_65_g', 'inw65_b'), ('inwoner_g', 'Inwoners_b'), ('medianIncome_g', 'medianInkomen_b'), ('p_benefits_g', 'p_benefits_b'), ('p_buildAfter2000_g', 'p_buildAfter2000_b'), ('p_buildBefore2000_g', 'p_buildBefore2000_b'), ('p_buyhouses_g', 'p_buyHouses_b'), ('p_nonWestern_g', 'p_nonWestern_b'), ('p_rentals_g', 'p_rental_houses_b'), ('p_socialHousing_g', 'p_socialHousing_b'), ('p_western_g', 'western_b')]
afs_haprak_g mi

p_socialHousing_g missing after: 8536
p_western_g missing before: 10606
p_western_g missing after: 8088
p_western_g missing before: 10580
p_western_g missing after: 8088
p_western_g missing before: 10567
p_western_g missing after: 8164
p_western_g missing before: 10547
p_western_g missing after: 8139
p_western_g missing before: 10511
p_western_g missing after: 8097
506 na's left of original 754, 0.00% of cols done, time spend: 10
492 na's left of original 754, 9.09% of cols done, time spend: 22
364 na's left of original 754, 18.18% of cols done, time spend: 34
319 na's left of original 754, 27.27% of cols done, time spend: 48
247 na's left of original 754, 36.36% of cols done, time spend: 55
218 na's left of original 754, 45.45% of cols done, time spend: 60
188 na's left of original 754, 54.55% of cols done, time spend: 67
155 na's left of original 754, 63.64% of cols done, time spend: 85
65 na's left of original 754, 72.73% of cols done, time spend: 100
16 na's left of original 754, 8

inw_1524_g missing before: 11750
inw_1524_g missing after: 8804
inw_2544_g missing before: 11247
inw_2544_g missing after: 8776
inw_2544_g missing before: 11233
inw_2544_g missing after: 8804
inw_2544_g missing before: 11230
inw_2544_g missing after: 8804
inw_2544_g missing before: 11235
inw_2544_g missing after: 8804
inw_2544_g missing before: 11231
inw_2544_g missing after: 8804
inw_4564_g missing before: 11204
inw_4564_g missing after: 8776
inw_4564_g missing before: 11191
inw_4564_g missing after: 8804
inw_4564_g missing before: 11168
inw_4564_g missing after: 8804
inw_4564_g missing before: 11140
inw_4564_g missing after: 8804
inw_4564_g missing before: 11103
inw_4564_g missing after: 8804
inw_65_g missing before: 10513
inw_65_g missing after: 8776
inw_65_g missing before: 10505
inw_65_g missing after: 8804
inw_65_g missing before: 10509
inw_65_g missing after: 8804
inw_65_g missing before: 10509
inw_65_g missing after: 8804
inw_65_g missing before: 10510
inw_65_g missing after: 8

0 na's left of original 1140, 93.33% of cols done, time spend: 320
total time: 320
1155 na's left of original 2615, 0.00% of cols done, time spend: 93
948 na's left of original 2615, 5.56% of cols done, time spend: 103
832 na's left of original 2615, 11.11% of cols done, time spend: 111
820 na's left of original 2615, 16.67% of cols done, time spend: 125
781 na's left of original 2615, 22.22% of cols done, time spend: 131
701 na's left of original 2615, 27.78% of cols done, time spend: 139
690 na's left of original 2615, 33.33% of cols done, time spend: 152
442 na's left of original 2615, 38.89% of cols done, time spend: 161
407 na's left of original 2615, 44.44% of cols done, time spend: 170
391 na's left of original 2615, 50.00% of cols done, time spend: 174
220 na's left of original 2615, 55.56% of cols done, time spend: 183
215 na's left of original 2615, 61.11% of cols done, time spend: 190
161 na's left of original 2615, 66.67% of cols done, time spend: 199
149 na's left of origi

373 na's left of original 403, 0.00% of cols done, time spend: 6
349 na's left of original 403, 10.00% of cols done, time spend: 9
268 na's left of original 403, 20.00% of cols done, time spend: 15
187 na's left of original 403, 30.00% of cols done, time spend: 20
147 na's left of original 403, 40.00% of cols done, time spend: 23
143 na's left of original 403, 50.00% of cols done, time spend: 27
142 na's left of original 403, 60.00% of cols done, time spend: 28
10 na's left of original 403, 70.00% of cols done, time spend: 33
1 na's left of original 403, 80.00% of cols done, time spend: 39
0 na's left of original 403, 90.00% of cols done, time spend: 41
total time: 41
400 na's left of original 430, 0.00% of cols done, time spend: 6
367 na's left of original 430, 10.00% of cols done, time spend: 9
317 na's left of original 430, 20.00% of cols done, time spend: 13
245 na's left of original 430, 30.00% of cols done, time spend: 18
201 na's left of original 430, 40.00% of cols done, time s

...

inw_014_g missing after: 8739
inw_1524_g missing before: 12725
inw_1524_g missing after: 8198
inw_1524_g missing before: 12684
inw_1524_g missing after: 8198
inw_1524_g missing before: 12644
inw_1524_g missing after: 8198
inw_1524_g missing before: 12571
inw_1524_g missing after: 8911
inw_1524_g missing before: 12495
inw_1524_g missing after: 8739
inw_2544_g missing before: 11811
inw_2544_g missing after: 8198
inw_2544_g missing before: 11761
inw_2544_g missing after: 8198
inw_2544_g missing before: 11700
inw_2544_g missing after: 8198
inw_2544_g missing before: 11640
inw_2544_g missing after: 8911
inw_2544_g missing before: 11615
inw_2544_g missing after: 8739
inw_4564_g missing before: 12836
inw_4564_g missing after: 8198
inw_4564_g missing before: 12773
inw_4564_g missing after: 8198
inw_4564_g missing before: 12705
inw_4564_g missing after: 8198
inw_4564_g missing before: 12637
inw_4564_g missing after: 8911
inw_4564_g missing before: 12563
inw_4564_g missing after: 8739
inw_65_g m

1267 na's left of original 4526, 23.53% of cols done, time spend: 276
1183 na's left of original 4526, 29.41% of cols done, time spend: 300
822 na's left of original 4526, 35.29% of cols done, time spend: 380
774 na's left of original 4526, 41.18% of cols done, time spend: 405
742 na's left of original 4526, 47.06% of cols done, time spend: 444
728 na's left of original 4526, 52.94% of cols done, time spend: 521
655 na's left of original 4526, 58.82% of cols done, time spend: 545
295 na's left of original 4526, 64.71% of cols done, time spend: 596
279 na's left of original 4526, 70.59% of cols done, time spend: 619
267 na's left of original 4526, 76.47% of cols done, time spend: 687
17 na's left of original 4526, 82.35% of cols done, time spend: 720
10 na's left of original 4526, 88.24% of cols done, time spend: 758
0 na's left of original 4526, 94.12% of cols done, time spend: 807
total time: 807
4313 na's left of original 6790, 0.00% of cols done, time spend: 279
4198 na's left of or

p_socialHousing_g missing after: 6868
p_socialHousing_g missing before: 9558
p_socialHousing_g missing after: 6868
p_socialHousing_g missing before: 9551
p_socialHousing_g missing after: 6868
p_western_g missing before: 9376
p_western_g missing after: 6657
p_western_g missing before: 9381
p_western_g missing after: 6657
p_western_g missing before: 9361
p_western_g missing after: 6657
p_western_g missing before: 9355
p_western_g missing after: 6663
p_western_g missing before: 9343
p_western_g missing after: 6663
814 na's left of original 1278, 0.00% of cols done, time spend: 17
744 na's left of original 1278, 8.33% of cols done, time spend: 32
657 na's left of original 1278, 16.67% of cols done, time spend: 42
603 na's left of original 1278, 25.00% of cols done, time spend: 59
433 na's left of original 1278, 33.33% of cols done, time spend: 80
193 na's left of original 1278, 41.67% of cols done, time spend: 97
146 na's left of original 1278, 50.00% of cols done, time spend: 105
142 na's

medianIncome_g missing before: 23406
medianIncome_g missing after: 22299
p_benefits_g missing before: 20513
p_benefits_g missing after: 18224
p_benefits_g missing before: 20500
p_benefits_g missing after: 18213
p_benefits_g missing before: 20468
p_benefits_g missing after: 18168
p_benefits_g missing before: 20463
p_benefits_g missing after: 18142
p_benefits_g missing before: 20441
p_benefits_g missing after: 18154
p_buildAfter2000_g missing before: 20513
p_buildAfter2000_g missing after: 18387
p_buildAfter2000_g missing before: 20500
p_buildAfter2000_g missing after: 18376
p_buildAfter2000_g missing before: 20468
p_buildAfter2000_g missing after: 18379
p_buildAfter2000_g missing before: 20463
p_buildAfter2000_g missing after: 18355
p_buildAfter2000_g missing before: 20441
p_buildAfter2000_g missing after: 18296
p_buildBefore2000_g missing before: 20513
p_buildBefore2000_g missing after: 18387
p_buildBefore2000_g missing before: 20500
p_buildBefore2000_g missing after: 18376
p_buildBefo

0 na's left of original 2846, 94.44% of cols done, time spend: 436
total time: 436
5130 na's left of original 5304, 0.00% of cols done, time spend: 6
2664 na's left of original 5304, 4.76% of cols done, time spend: 110
2152 na's left of original 5304, 9.52% of cols done, time spend: 118
1852 na's left of original 5304, 14.29% of cols done, time spend: 124
1712 na's left of original 5304, 19.05% of cols done, time spend: 130
1692 na's left of original 5304, 23.81% of cols done, time spend: 138
1622 na's left of original 5304, 28.57% of cols done, time spend: 143
1607 na's left of original 5304, 33.33% of cols done, time spend: 148
1551 na's left of original 5304, 38.10% of cols done, time spend: 153
1541 na's left of original 5304, 42.86% of cols done, time spend: 161
1383 na's left of original 5304, 47.62% of cols done, time spend: 166
1343 na's left of original 5304, 52.38% of cols done, time spend: 171
1334 na's left of original 5304, 57.14% of cols done, time spend: 175
1320 na's le

In [17]:
a[4].woonfunctiesum.sum()

76518.0

In [98]:
b = fill_final_na(a)

(3230, 46)
(38, 46)
1764 na's left of original 1802, 0.00% of cols done, time spend: 11


KeyboardInterrupt: 

In [57]:
a.loc[ 522]

bijeenkomstfunctiesum                                                           0.0
geometry                          POLYGON ((145900 418200, 146000 418200, 146000...
gezondheidszorgfunctiesum                                                       0.0
industriefunctiesum                                                             0.0
kantoorfunctiesum                                                               0.0
logiesfunctiesum                                                                0.0
onderwijsfunctiesum                                                             0.0
sportfunctiesum                                                                 0.0
winkelfunctiesum                                                                0.0
woonfunctiesum                                                                  1.0
oppervlakteVerblijfsobjectmean                                                140.0
inwoner_g                                                                   

In [22]:
b = a[a.woonfunctiesum > 0]

In [64]:
(a.isna().sum(axis=1) == 0).any()

AttributeError: 'NoneType' object has no attribute 'isna'

In [58]:
a.isna().sum(axis = 0)
a.mean_WOZ_b.isna().sum()

9816

In [53]:
a.shape

(13524, 74)

In [21]:
a

,bijeenkomstfunctiesum,geometry,gezondheidszorgfunctiesum,industriefunctiesum,kantoorfunctiesum,logiesfunctiesum,onderwijsfunctiesum,sportfunctiesum,winkelfunctiesum,woonfunctiesum,...,afs_vo_g,afs_oprit_g,afs_train_g,afs_transit_g,mean_WOZ,PLaagste40Inkomen,PHoogste20Inkomen,bouwjaarmean,bouwjaarmin,bouwjaarmax
85,0.0,"POLYGON ((235300.000 587200.000, 235400.000 58...",0.0,0.0,0.0,0.0,0.0,1.0,0.0,3.0,...,3.4,1.0,4.8,7.0,214.0,35.8,19.3,1945.250000,1928.0,1981.0
86,1.0,"POLYGON ((235400.000 587200.000, 235500.000 58...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,...,3.4,1.0,4.8,7.0,214.0,35.8,19.3,1970.543478,1932.0,2003.0
88,0.0,"POLYGON ((235600.000 587200.000, 235700.000 58...",0.0,0.0,0.0,0.0,0.0,0.0,1.0,5.0,...,3.4,1.0,4.8,7.0,214.0,35.8,19.3,1942.272727,1833.0,2000.0
100,0.0,"POLYGON ((236800.000 587200.000, 236900.000 58...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,4.2,2.3,5.6,7.7,NaN,NaN,NaN,1961.000000,1915.0,2007.0
118,0.0,"POLYGON ((238600.000 587200.000, 238700.000 58...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,7.3,2.7,8.7,10.8,NaN,34.9,21.3,1955.000000,1955.0,1955.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,0.0,"POLYGON ((237700.000 577500.000, 237800.000 57...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,19.0,...,2.4,0.9,1.3,7.4,NaN,37.9,22.4,1977.437500,1907.0,2002.0
13496,0.0,"POLYGON ((237800.000 577500.000, 237900.000 57...",0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,...,2.4,0.9,1.3,7.4,NaN,37.9,22.4,1984.733333,1948.0,1997.0
13497,0.0,"POLYGON ((237900.000 577500.000, 238000.000 57...",0.0,1.0,0.0,0.0,0.0,0.0,0.0,18.0,...,2.4,0.9,1.3,7.4,NaN,37.9,22.4,1980.666667,1915.0,2000.0
13498,0.0,"POLYGON ((238000.000 577500.000, 238100.000 57...",0.0,0.0,1.0,0.0,0.0,0.0,0.0,15.0,...,2.4,0.9,1.3,7.4,NaN,37.9,22.4,1976.611111,1913.0,1996.0


In [73]:
.01a.isna().sum(\)

bijeenkomstfunctiesum             0
gezondheidszorgfunctiesum         0
industriefunctiesum               0
kantoorfunctiesum                 0
logiesfunctiesum                  0
onderwijsfunctiesum               0
sportfunctiesum                   0
winkelfunctiesum                  0
woonfunctiesum                    0
oppervlakteVerblijfsobjectmean    0
inwoner_g                         0
geboorte_g                        0
inw_014_g                         0
inw_1524_g                        0
inw_2544_g                        0
inw_4564_g                        0
inw_65_g                          0
p_western_g                       0
p_nonWestern_g                    0
p_buyhouses_g                     0
p_rentals_g                       0
p_socialHousing_g                 0
medianIncome_g                    0
avg_electricity_g                 0
avg_gas_g                         0
p_benefits_g                      0
afs_haprak_g                      0
afs_ziek_g                  

In [47]:
for j,i in a.max().items():
    print(j,i)

Unnamed: 0 12562.0
Unnamed: 0.1 12562.0
Unnamed: 0.1.1 2911096.0
bijeenkomstfunctiesum 12.0
gezondheidszorgfunctiesum 31.0
industriefunctiesum 20.0
kantoorfunctiesum 17.0
logiesfunctiesum 3.0
onderwijsfunctiesum 4.0
sportfunctiesum 1.0
winkelfunctiesum 38.0
woonfunctiesum 201.0
oppervlakteVerblijfsobjectmean 34964.5
bouwjaarmean 2014.0
bouwjaarmin 2014.0
bouwjaarmax 2014.0
y 1.0
inwoner_g 270.0
geboorte_g 10.0
inw_014_g 55.0
inw_1524_g 215.0
inw_2544_g 170.0
inw_4564_g 175.0
inw_65_g 100.0
p_NL 60.0
p_western_g 70.0
p_nonWestern_g 100.0
p_buyhouses_g 100.0
p_rentals_g 100.0
p_socialHousing_g 190.0
avg_electricity_g 5320.0
avg_gas_g 2530.0
p_benefits_g 1.0
afs_haprak_g 3.5
afs_ziek_g 10.0
afs_superm_g 3.3
afs_vo_g 8.6
afs_oprit_g 4.6
afs_train_g 9.6
afs_transit_g 11.5
p_buildBefore2000_g 1.0002450686192137
p_buildAfter2000_g 0.5001917459216476
afs_huisarts_b 5.2
afs_ziekenhuis_b 10.9
afs_supermarkt_b 4.9
afs_vo_b 10.1
afs_oprit_b 4.5
afs_train_b 11.1
afs_transit_b 13.9
Inwoners_b 8055.0

C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


In [27]:
a.min()

C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  """Entry point for launching an IPython kernel.


Unnamed: 0                                                           5
Unnamed: 0.1                                                         5
C28992R100                                                  E1425N4094
Unnamed: 0.1.1                                                 2746653
geometry             POLYGON ((142500 409500, 142600 409500, 142600...
                                           ...                        
avg_gas_b                                                        280.0
medianInkomen_b                                                   19.3
PLaagste40Inkomen                                                 16.5
PHoogste20Inkomen                                                  2.2
p_benefits_b                                                       0.0
Length: 73, dtype: object

In [46]:
(a.p_benefits_g <0).sum()

0

In [37]:
len(a)

3558

In [12]:
b.isna().sum()

Unnamed: 0                0
Unnamed: 0.1              0
C28992R100                0
Unnamed: 0.1.1            0
geometry                  0
                       ... 
avg_gas_b              3558
medianInkomen_b        3558
PLaagste40Inkomen_b    3558
PHoogste20Inkomen_b    3558
pBenefits_b            3558
Length: 74, dtype: int64

In [3]:
data2015 = pd.read_csv("BRgrid2015.csv")
data2016 = pd.read_csv("BRgrid2016.csv")
data2017 = pd.read_csv("BRgrid2017.csv")
data2018 = pd.read_csv("BRgrid2018.csv")
data2019 = pd.read_csv("BRgrid2019.csv")

In [4]:
data2015.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2016.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2017.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2018.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)
data2019.rename(columns = {"p_buildBefore2000": "p_buildBefore2000_g", "p_buildAfter2000": "p_buildAfter2000_g", "p_wester_g" : "p_western_g", "afs_school_b" : "afs_vo_b", "afs_treins_g" : "afs_train_g", "nonWestern_b" : "p_nonWestern_b", "mean_WOZ_b" : "mean_WOZ", "pBenefits_b" : "p_benefits_b", "p_NL_g" : "p_NL", "PHoogste20Inkomen_b" : "PHoogste20Inkomen", "PLaagste40Inkomen_b" : "PLaagste40Inkomen" }, inplace = True)

In [5]:
data2015 = data2015.replace({-99997 : np.NaN, np.inf : np.NaN})
data2016 = data2016.replace({-99997 : np.NaN, np.inf : np.NaN})
data2017 = data2017.replace({-99997 : np.NaN, np.inf : np.NaN})
data2018 = data2018.replace({-99997 : np.NaN, np.inf : np.NaN})
data2019 = data2019.replace({-99997 : np.NaN, np.inf : np.NaN})


In [6]:
b15 = data2015[data2015.woonfunctie>0]
b16 = data2016[data2016.woonfunctie>0]
b17 = data2017[data2017.woonfunctie>0]
b18 = data2018[data2018.woonfunctie>0]
b19 = data2019[data2019.woonfunctie>0]

In [7]:
nadf["data2015"] = data2015.isna().sum() / len(data2015)
nadf["residential2015"] = b15.isna().sum() / len(b15)

nadf["data2016"] = data2016.isna().sum() / len(data2015)
nadf["residential2016"] = b16.isna().sum() / len(b15)

nadf["data2017"] = data2017.isna().sum() / len(data2015)
nadf["residential2017"] = b17.isna().sum() / len(b15)

nadf["data2018"] = data2018.isna().sum() / len(data2015)
nadf["residential2018"] = b18.isna().sum() / len(b15)

nadf["data2019"] = data2019.isna().sum() / len(data2015)
nadf["residential2019"] = b19.isna().sum() / len(b15)

In [8]:
def grid_filler(data, grid_column, buurt_column):
    print(grid_column + " missing before: {}".format(data[grid_column].isna().sum()))
    missing_grid_data = data[grid_column].isna().index.values
    data.loc[missing_grid_data, grid_column] = data.loc[missing_grid_data, buurt_column]
    print(grid_column + " missing after: {}".format(data[grid_column].isna().sum()))
    return data[grid_column]

In [9]:
grid_variables = []
neighborhood_variables = []
for var in data2015.columns:
    if var.endswith("_g"):
        grid_variables.append(var)
    elif var.endswith("_b"):
        neighborhood_variables.append(var)
        
grid_variables.sort(key=str.lower)
neighborhood_variables.sort(key=str.lower)

In [10]:
ls = list()
for i, j in zip(grid_variables, neighborhood_variables):
    ls.append((i,j))
print(ls)

[('afs_haprak_g', 'afs_huisarts_b'), ('afs_oprit_g', 'afs_oprit_b'), ('afs_superm_g', 'afs_supermarkt_b'), ('afs_train_g', 'afs_train_b'), ('afs_transit_g', 'afs_transit_b'), ('afs_vo_g', 'afs_vo_b'), ('afs_ziek_g', 'afs_ziekenhuis_b'), ('avg_electricity_g', 'avg_electicity_b'), ('avg_gas_g', 'avg_gas_b'), ('geboorte_g', 'birth_b'), ('inw_014_g', 'inw14_b'), ('inw_1524_g', 'inw24_b'), ('inw_2544_g', 'inw44_b'), ('inw_4564_g', 'inw64_b'), ('inw_65_g', 'inw65_b'), ('inwoner_g', 'Inwoners_b'), ('medianIncome_g', 'medianInkomen_b'), ('p_benefits_g', 'p_benefits_b'), ('p_buildAfter2000_g', 'p_buildAfter2000_b'), ('p_buildBefore2000_g', 'p_buildBefore2000_b'), ('p_buyhouses_g', 'p_buyHouses_b'), ('p_nonWestern_g', 'p_nonWestern_b'), ('p_rentals_g', 'p_rental_houses_b'), ('p_socialHousing_g', 'p_socialHousing_b'), ('p_western_g', 'western_b')]


In [11]:
for i in ls:
    data2015[i[0]] = grid_filler(data2015, i[0], i[1])
    data2016[i[0]] = grid_filler(data2016, i[0], i[1])
    data2017[i[0]] = grid_filler(data2017, i[0], i[1])
    data2018[i[0]] = grid_filler(data2018, i[0], i[1])
    data2019[i[0]] = grid_filler(data2019, i[0], i[1])
    

afs_haprak_g missing before: 15560
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 15546
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 15518
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 15510
afs_haprak_g missing after: 12807
afs_haprak_g missing before: 19324
afs_haprak_g missing after: 12807
afs_oprit_g missing before: 15555
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 15537
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 15518
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 15510
afs_oprit_g missing after: 12807
afs_oprit_g missing before: 19324
afs_oprit_g missing after: 12807
afs_superm_g missing before: 15555
afs_superm_g missing after: 12807
afs_superm_g missing before: 15537
afs_superm_g missing after: 12807
afs_superm_g missing before: 15518
afs_superm_g missing after: 12807
afs_superm_g missing before: 15510
afs_superm_g missing after: 12807
afs_superm_g missing before: 19324
afs_super

In [12]:
def visualize_NANs(data):
    data["n_missing"] = data.isna().sum(axis=1)
    data.plot(figsize = (40,40), alpha=0.5, edgecolor = "b", column='n_missing',legend=True)

In [13]:
var = ['geometry', 'bijeenkomstfunctie',
       'celfunctie', 'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 'BU_CODE',
       'y', 'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_NL', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g', 'mean_WOZ', 
        'PLaagste40Inkomen', 'PHoogste20Inkomen']

In [14]:
t15 = data2015[var]
t16 = data2016[var]
t17 = data2017[var]
t18 = data2018[var]
t19 = data2019[var]


In [15]:
t15["n_missing"] = t15.isna().sum(axis=1)
t16["n_missing"] = t16.isna().sum(axis=1)
t17["n_missing"] = t17.isna().sum(axis=1)
t18["n_missing"] = t18.isna().sum(axis=1)
t19["n_missing"] = t19.isna().sum(axis=1)

t15b = t15[(t15.woonfunctie > 0) & (t15.n_missing < 15)]
t16b = t16[(t16.woonfunctie > 0) & (t16.n_missing < 15)]
t17b = t17[(t17.woonfunctie > 0) & (t17.n_missing < 15)]
t18b = t18[(t18.woonfunctie > 0) & (t18.n_missing < 15)]
t19b = t19[(t19.woonfunctie > 0) & (t19.n_missing < 15)]

t15b = gpd.GeoDataFrame(t15b)
t15b.geometry = t15b.geometry.apply(shapely.wkt.loads)

t16b = gpd.GeoDataFrame(t16b)
t16b.geometry = t16b.geometry.apply(shapely.wkt.loads)

t17b = gpd.GeoDataFrame(t17b)
t17b.geometry = t17b.geometry.apply(shapely.wkt.loads)

t18b = gpd.GeoDataFrame(t18b)
t18b.geometry = t18b.geometry.apply(shapely.wkt.loads)

t19b = gpd.GeoDataFrame(t19b)
t19b.geometry = t19b.geometry.apply(shapely.wkt.loads)


C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
C:\Users\Gebruiker\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See th

In [16]:
data2015.columns

Index(['C28992R100', 'Unnamed: 0', 'geometry', 'bijeenkomstfunctie',
       'celfunctie', 'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 'BU_CODE',
       'y', 'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_NL', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g',
       'afs_huisarts_b', 'afs_ziekenhuis_b', 'afs_supermarkt_b', 'afs_vo_b',
       'afs_oprit_b', 'afs_train_b', 'afs_transit_b', 'Inwoners_b', 'inw14_b',
       'inw24_b', 'inw44_b', 'inw64_b', 'inw65_b', 'western_b',
       'p_nonWestern_b', 'birth_b

In [17]:
var2 = ['bijeenkomstfunctie', 'geometry',
       'celfunctie', 'gezondheidszorgfunctie', 'industriefunctie',
       'kantoorfunctie', 'logiesfunctie', 'onderwijsfunctie', 'sportfunctie',
       'winkelfunctie', 'woonfunctie', 'oppervlakteVerblijfsobject', 
        'inwoner_g', 'geboorte_g', 'inw_014_g', 'inw_1524_g', 'inw_2544_g',
       'inw_4564_g', 'inw_65_g', 'p_western_g', 'p_nonWestern_g',
       'p_buyhouses_g', 'p_rentals_g', 'p_socialHousing_g', 'medianIncome_g',
       'avg_electricity_g', 'avg_gas_g', 'p_benefits_g', 'afs_haprak_g',
       'afs_ziek_g', 'afs_superm_g', 'afs_vo_g', 'afs_oprit_g', 'afs_train_g',
       'afs_transit_g', 'p_buildBefore2000_g', 'p_buildAfter2000_g', 'mean_WOZ', 
        'PLaagste40Inkomen', 'PHoogste20Inkomen']

In [18]:
t15b = t15b[var2]
t15b = gpd.GeoDataFrame(t15b)
t16b = t16b[var2]
t16b = gpd.GeoDataFrame(t16b)
t17b = t17b[var2]
t17b = gpd.GeoDataFrame(t17b)
t18b = t18b[var2]
t18b = gpd.GeoDataFrame(t18b)
t19b = t19b[var2]
t19b = gpd.GeoDataFrame(t19b)



In [19]:
def fill_final_na(data):
    t0 = time.time()
    total_na = data.isna().sum().sum()
    all_geom = data[["geometry", "sportfunctie"]]

    data = data.drop(["geometry"], axis = 1)
    incomplete = data[data.isnull().any(axis=1)]
    complete = gpd.GeoDataFrame(data[~data.isnull().any(axis=1)]) # store the rows that have no na values
    
    
    
    # create list of combinatoins of missing columns
    inc_list = list()
    for _, i in incomplete.iterrows():
        if not tuple(i.index[i.isna()]) in inc_list:
            inc_list.append(tuple(i.index[i.isna()]))
            
#   Create weightsmatrix and transform to row_standardized            
#     w_all = weights.distance.KNN.from_dataframe(all_geom, k = 4, silence_warnings = True)
#     w_all.transform = 'r'
    
    
    columns = data.columns
    
    # loop over the missing column combinations
    def train_creater(inp, w, train = "train", test = False):
        """
        returns: new dataframe including the neighbors data
        """
        # Get neigbors per index
        neighbors = [ w.neighbors[k] for k in inp.index.values.tolist()]
        if test:
            
            df = pd.concat([inp, train])
            
            rows = [ np.concatenate(df.loc[j].values) for j in neighbors] 
            distances = [list(all_geom.loc[neigh].distance(all_geom.loc[df.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]
        else:
            # get rows of the neigbors, tranform to np and concatenate them
            rows = [ np.concatenate(inp.loc[j].values) for j in neighbors] 
            distances = [list(all_geom.loc[neigh].distance(all_geom.loc[inp.iloc[i].name].geometry).values) for i, neigh in enumerate(neighbors)]
        
        
        return pd.concat([pd.DataFrame(rows).set_index(inp.index), inp, pd.DataFrame(distances).set_index(inp.index) ], axis=1)
        
    for i, cols in enumerate(inc_list):
        cols = list(cols)
        x_train = complete.drop(cols, axis = 1) # only keep the cols to trained upon
        y_train = complete[cols] # y has the na values to be predicted
       
        w_train  = weights.distance.KNN.from_dataframe(all_geom.loc[x_train.index], k = 4, silence_warnings = True)
        w_train.transform = 'r'
        
        X_train = train_creater(x_train, w_train)
        
        # Voor x predict only keep the rows where there are only NA's in cols
        pred = incomplete[(incomplete[cols].isna().all(axis = 1) & (incomplete[columns.drop(cols)].notna().all(axis = 1)))]
        X_pred = pred.drop(cols, axis = 1)

        # create prediction weights of missing rows as well as known rows
        w_pred =  weights.distance.KNN.from_dataframe(all_geom.loc[X_train.index.append(X_pred.index)], k = 4, silence_warnings = True)
        w_pred.transform = 'r'
        X_pred = train_creater(X_pred, w_pred, x_train, True)
        
        # loop over each missing variable in cols and create model per 
        for col in cols:
            
            clf = RandomForestRegressor(max_depth=6, random_state=0)
            clf.fit(X_train, y_train[col].values.ravel())    
            y_pred = clf.predict(X_pred)
            incomplete.loc[X_pred.index, col] = y_pred
            
        print("{} na's left of original {}, {:.2f}% of cols done, time spend: {:.0f}".format(incomplete.isna().sum().sum(),  total_na, i / len(inc_list) * 100, time.time() - t0))
    print("total time: {:.0f}".format(time.time() - t0))
    return pd.concat([incomplete, complete])
            


In [20]:

t15b = fill_final_na(t15b)
t16b = fill_final_na(t16b)
t17b = fill_final_na(t17b)
t18b = fill_final_na(t18b)
t19b = fill_final_na(t19b)

C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


788 na's left of original 885, 0.00% of cols done, time spend: 12


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

592 na's left of original 885, 14.29% of cols done, time spend: 38


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


...

C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

180 na's left of original 2409, 75.00% of cols done, time spend: 84


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#r

0 na's left of original 2409, 87.50% of cols done, time spend: 109
total time: 109


C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
C:\Users\Gebruiker\anaconda3\lib\site-packages\pandas\core\indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)


In [117]:
t15b

,bijeenkomstfunctie,celfunctie,gezondheidszorgfunctie,industriefunctie,kantoorfunctie,logiesfunctie,onderwijsfunctie,sportfunctie,winkelfunctie,woonfunctie,...,afs_superm_g,afs_vo_g,afs_oprit_g,afs_train_g,afs_transit_g,p_buildBefore2000_g,p_buildAfter2000_g,mean_WOZ,PLaagste40Inkomen,PHoogste20Inkomen
100,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3.3,4.3,2.1,5.7,7.9,94.0,6.0,391.764651,34.453318,37.313453
345,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.3,4.3,2.1,5.7,7.9,94.0,6.0,393.048448,34.618565,38.903016
349,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,...,3.3,4.3,2.1,5.7,7.9,94.0,6.0,390.136723,34.509750,38.094073
611,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,...,3.2,3.5,1.4,4.4,6.5,88.0,12.0,283.960709,35.672029,35.528626
625,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,...,3.2,3.5,1.4,4.4,6.5,88.0,12.0,284.419384,35.615597,37.537007
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13495,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,18.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000
13496,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,14.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000
13497,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,17.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000
13498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,15.0,...,0.7,1.4,0.9,1.3,7.3,92.0,8.0,189.000000,37.100000,23.800000


In [28]:
t15b = t15b.join(data2015[["C28992R100", "geometry", "y", "BU_CODE"]])
t16b = t16b.join(data2016[["C28992R100", "geometry", "y", "BU_CODE"]])
t17b = t17b.join(data2017[["C28992R100", "geometry", "y", "BU_CODE"]])
t18b = t18b.join(data2018[["C28992R100", "geometry", "y", "BU_CODE"]])
t19b = t19b.join(data2019[["C28992R100", "geometry", "y", "BU_CODE"]])

In [29]:
t15b.to_csv("BRfilled_data15.csv")
t16b.to_csv("BRfilled_data16.csv")
t17b.to_csv("BRfilled_data17.csv")
t18b.to_csv("BRfilled_data18.csv")
t19b.to_csv("BRfilled_data19.csv")

In [26]:
t15b.shape

(4404, 39)